In [ ]:
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import xlrd
import os
from collections import OrderedDict

In [ ]:
#Excel file you'd use
XLSX_FILENAME = r'D:\Documents\Source\Anaconda\sklearn\graphGenerator\2021-01-05 Total KD result.xlsx'

#Set available data group and graph color for each data group
availableDataGroups = {}

availableDataGroups["Genotype"] = ('WT-Untreated', 'HT-Untreated', 'Homo-Untreated', 'WT-OCT4', 'HT-OCT4', 'Homo-OCT4', 'WT-GFP', 'HT-GFP', 'Homo-GFP', 'WT', 'WT-ABE', 'HT-ABE', 'Homo-ABE')
availableDataGroups["Gender"] = ("Male", "Female")


colorsavailable = ("b", "r")

genotypesavailable = ('WT-Untreated', 'HT-Untreated', 'Homo-Untreated', 'WT-OCT4', 'HT-OCT4', 'Homo-OCT4', 'WT-GFP', 'HT-GFP', 'Homo-GFP', 'WT', 'WT-ABE', 'HT-ABE', 'Homo-ABE')
colorsavailable = ("dimgray","darkgray","lightgray","darkslategray","teal","darkturquoise","darkolivegreen","olivedrab","yellowgreen","tomato","maroon","brown","lightcoral")

colorsdict = {}
for genotype, color in zip(genotypesavailable, colorsavailable):
    colorsdict[genotype] = color

In [ ]:
#Get sheetnames
xls = xlrd.open_workbook(XLSX_FILENAME, on_demand = True)
sheetnames = xls.sheet_names()

#Get dataframe to dictionary. uses sheetnames as keys
dfs = {}
for sheetname in sheetnames:
    dfs[sheetname] = pd.read_excel(XLSX_FILENAME, header=1, sheet_name = sheetname)

In [ ]:
#Returns Split Dataframes
def getSplitDataframe(df):
    
    dfParts = []
    dfPartsColumn = []
    
    emptyColumns = df.columns[df.isnull().all(axis = 0)].tolist()
    emptyColumnsIndex = df.columns.get_indexer(emptyColumns)

    priorindex = 0
    
    for index in emptyColumnsIndex:
        dfPartsColumn.append(df.columns[priorindex:index])
        priorindex = index + 1

    dfPartsColumn.append(df.columns[priorindex:])

    for i in range(len(dfPartsColumn)):
        dfParts.append(df.loc[:,dfPartsColumn[i]])

    for dfPart in dfParts[1:]:
        for i in range(len(dfPart.columns)):
            dotindex = dfPart.iloc[:,i].name.find(".")
            dfPart.rename(columns = {dfPart.iloc[:,i].name : dfPart.iloc[:,i].name[0:dotindex if dotindex != -1 else None]}, inplace = True)

    return dfParts

In [ ]:
def exportGraphPng(sheetname):
    os.makedirs(os.path.join(os.path.curdir, "graphs"), exist_ok = True)
    for dfPart in splitDfsDictOfList[sheetname]:
        try:
            genotypes = dfPart["Group"].unique()[~np.isnan(dfPart["Group"].unique())]
            genotypesstring = dfPart.groupby("Group")["Genotype"].first().values

            week = next(iter([col for col in dfPart.columns.to_list() if "wks" in col]), None)
            sems = np.nan_to_num(dfPart.groupby("Group")["Average"].sem().to_list())
            ns = np.nan_to_num(dfPart.groupby("Group")["Average"].count().to_list())
            averages = np.nan_to_num(dfPart.groupby("Group")["Average"].mean().to_list())

            colorsused = []
            for genotypestring in genotypesstring:
                colorsused.append(colorsdict[genotypestring])
            
            fig, ax = plt.subplots(figsize = (2.4*len(genotypes),8))
            bars = ax.bar(genotypesstring, averages, yerr = sems, capsize = 5, edgecolor = "black", color = colorsused, zorder = 3)
            ax.set_title(sheetname + " - " + week)
            ax.set_ylabel("Rearing Times")
            ax.set_ylim(0, max(averages + sems) * 1.2)
            ax.grid(axis = "y", zorder = 0)
            ax.spines["left"].set_linewidth(2)
            ax.spines["bottom"].set_linewidth(2)
            ax.spines["top"].set_linewidth(0)
            ax.spines["right"].set_linewidth(0)

            SMALL_SIZE = 12
            MEDIUM_SIZE = 15
            BIGGER_SIZE = 20

            plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
            plt.rc('axes', titlesize=BIGGER_SIZE)     # fontsize of the axes title
            plt.rc('axes', labelsize=BIGGER_SIZE)    # fontsize of the x and y labels
            plt.rc('xtick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
            plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
            plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title
            
            for index, rect in enumerate(bars):
                barlen = np.nan_to_num(averages[index])
                maxbarlen = max(np.nan_to_num(averages))
                errlen = np.nan_to_num(sems[index])
                if barlen != 0:
                    ax.text(rect.get_x() + rect.get_width() / 2., (barlen + errlen) + maxbarlen / 7, "N = %s" %ns[index] , ha='center', va='center')
                    ax.text(rect.get_x() + rect.get_width() / 2., (barlen + errlen) + maxbarlen / 15., "mean = %.2f" %averages[index], ha='center', va='center')
                    rect.set_linewidth(2)

            fig.savefig(os.path.join(os.path.curdir, "graphs", sheetname + "_" + week))
        except Exception as e:
            print("For sheetname %s: form not appropriate\n" %sheetname, e)

In [ ]:
splitDfsDictOfList = OrderedDict()
for sheetname in sheetnames:
    splitDfsDictOfList.update({sheetname : getSplitDataframe(dfs[sheetname])})

In [ ]:
for sheetname in sheetnames:
    exportGraphPng(sheetname)

In [ ]:
# dfParts = []
# dfPartsColumn = []

# for sheetindex, sheetname in enumerate(sheetnames):
#     df = dfs[sheetname]

#     columnWithNaNValue = dfs[sheetname].columns[dfs[sheetname].isnull().all(axis = 0)].tolist()
#     columnIndexWithNaNValue = dfs[sheetname].columns.get_indexer(columnWithNaNValue)

#     priorindex = 0
#     for count, index in enumerate(columnIndexWithNaNValue):
#         dfPartsColumn.append(dfs[sheetname].columns[priorindex:index])
#         priorindex = index + 1

#     dfPartsColumn.append(dfs[sheetname].columns[priorindex:])

#     for i in range(len(dfPartsColumn)):
#         dfParts.append(dfs[sheetname].loc[:,dfPartsColumn[i]])

#     for dfPart in dfParts[1:]:
#         for i in range(len(dfPart.columns)):
#             indexofdot = dfPart.iloc[:,i].name.find(".")
#             dfPart.rename(columns = {dfPart.iloc[:,i].name : dfPart.iloc[:,i].name[0:indexofdot if indexofdot != -1 else None]}, inplace = True)

In [ ]:
# os.makedirs(os.path.join(os.path.curdir, "graphs"), exist_ok = True)


# for dfPart in dfParts:
#     genotypes = dfPart["Group"].unique()[~np.isnan(dfPart["Group"].unique())]
#     genotypesstring = dfPart.groupby("Group")["Genotype"].first().values

#     week = [col for col in dfPart.columns.to_list() if "wks" in col][0]
#     sems = np.nan_to_num(dfPart.groupby("Group")["Average"].sem().to_list())
#     ns = np.nan_to_num(dfPart.groupby("Group")["Average"].count().to_list())
#     averages = np.nan_to_num(dfPart.groupby("Group")["Average"].mean().to_list())
#     colorsused = []
#     for genotypestring in genotypesstring:
#         colorsused.append(colorsdict[genotypestring])
    
#     fig, ax = plt.subplots(figsize = (2.5*len(genotypes),6))
#     bars = ax.bar(genotypesstring, averages, yerr = sems, capsize = 5, edgecolor = "black", color = colorsused, zorder = 3)
#     ax.set_title(sheetnames[sheetIndex] + " - " + week)
#     ax.set_ylabel(" Rearing Times")
#     ax.set_ylim(0, max(averages + sems) * 1.2)
#     ax.grid(axis = "y", zorder = 0)
#     ax.spines["left"].set_linewidth(2)
#     ax.spines["bottom"].set_linewidth(2)
#     ax.spines["top"].set_linewidth(0)
#     ax.spines["right"].set_linewidth(0)   

#     SMALL_SIZE = 12
#     MEDIUM_SIZE = 15
#     BIGGER_SIZE = 20

#     plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
#     plt.rc('axes', titlesize=BIGGER_SIZE)     # fontsize of the axes title
#     plt.rc('axes', labelsize=BIGGER_SIZE)    # fontsize of the x and y labels
#     plt.rc('xtick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
#     plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
#     plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title
    
#     for index, rect in enumerate(bars):
#         barlen = np.nan_to_num(averages[index])
#         maxbarlen = max(np.nan_to_num(averages))
#         errlen = np.nan_to_num(sems[index])
#         if barlen != 0:
#             ax.text(rect.get_x() + rect.get_width() / 2., (barlen + errlen) + maxbarlen / 7, "N = %s" %ns[index] , ha='center', va='center')
#             ax.text(rect.get_x() + rect.get_width() / 2., (barlen + errlen) + maxbarlen / 15., "mean = %.2f" %averages[index], ha='center', va='center')
#             rect.set_linewidth(2)

#     fig.set_size_inches(2.5*len(genotypes),6)
#     fig.savefig(os.path.join(os.path.curdir, "graphs", sheetnames[sheetIndex] + "_" + week))
#     plt.show()

In [ ]:
#top = tkinter.Tk()
#top.mainloop()

In [ ]:
os.path.join(os.path.curdir, "graphs")